In [2]:
import cv2
import numpy as np
import joblib
from sklearn.base import BaseEstimator, TransformerMixin

# --- Define any custom transformers used in the pipeline ---
class MeanCentering(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.mean_face = np.mean(X, axis=0)
        return self

    def transform(self, X):
        return X - self.mean_face

# --- Load the Haar Cascade face detector ---
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# --- Load the trained pipeline (update filename if needed) ---
pipeline = joblib.load("eigenface_pipeline.pkl")  # Or "eigenface_pipeline.pkl" if using that one

# --- Function to detect faces ---
def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces, gray

# --- Function to predict label for a face ---
def predict_label(face_gray):
    face_resized = cv2.resize(face_gray, (128, 128))
    face_flattened = face_resized.flatten().reshape(1, -1)
    label = pipeline.predict(face_flattened)[0]
    return label

# --- Initialize webcam ---
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    faces, gray = detect_faces(frame)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        label = predict_label(face)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Eigenface Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Press 'q' to quit.
